In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset
import numpy as np

In [3]:
from common import *

In [4]:
data = Dataset(name='Diagnosis', dirname='preprocess/Diagnosis/')

INFO - 2019-02-12 11:26:51,141 - Reading Structured data ...
INFO - 2019-02-12 11:26:51,238 - Reading Notes ...
INFO - 2019-02-12 11:27:01,141 - Stratifying ...


In [5]:
labellist = [x for x in data.dataframe.columns if x.startswith('y_')]
data.generate_labels(labellist, len(labellist), 'multilabel')

In [6]:
data.generate_encoded_field('gender_y', 'onehot')
data.generate_encoded_field('age_y', 'onehot')
data.generate_encoded_field('ethnicity_y', 'onehot')

In [7]:
features = [x for x in data.dataframe.columns if x.startswith('feature')]
for f in features :
    data.generate_encoded_field(f, 'trivial')
    
data.set_structured_params(regexs=[r'^feature', 'gender_y', 'age_y', 'ethnicity_y'])

In [8]:
from trainer import Trainer, Evaluator

In [9]:
from models.Vanilla import ClassificationTrainer as BasicCT
from models.Hierarchical import ClassificationTrainer as HierCT

In [10]:
from Experiments.experiments import experiments, hierarchical_experiments, structured_experiments

Basic Experiments
=================

In [11]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)
test_data = data.get_data('test', structured=True)

Pos Percentage [0.21358209 0.07716941 0.10244004 0.31971179 0.13028647 0.13014724
 0.20870897 0.07160013 0.26850917 0.31939852 0.09485189 0.19033033
 0.28253681 0.41581677 0.26697762 0.07180897 0.13039089 0.0897003
 0.04953183 0.04076021 0.08917818 0.14152947 0.18249852 0.14615893
 0.07797   ]
Pos Percentage [0.20548768 0.06969758 0.10194272 0.31744442 0.12237132 0.13458842
 0.2032846  0.06649309 0.25776087 0.31484078 0.0929301  0.19587422
 0.29541358 0.42339275 0.25475666 0.06789505 0.12096936 0.08531945
 0.0530743  0.03745243 0.0817144  0.1327859  0.17684759 0.13198478
 0.07790907]
Pos Percentage [0.20977494 0.06768559 0.10732281 0.32129661 0.12512597 0.12395029
 0.21229426 0.06902922 0.26603964 0.32935841 0.09103124 0.19146792
 0.28081962 0.42122943 0.26184078 0.07910648 0.12428619 0.08834397
 0.05424924 0.04165267 0.09271078 0.13671481 0.17954316 0.14124958
 0.08095398]


In [12]:
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

INFO - 2019-02-12 11:27:18,567 - Maximum Sentence Length 721825.000000 , 90 percentile length 16714.400000 ... 
INFO - 2019-02-12 11:27:18,573 - Pos Percentage of remaining data ... 
INFO - 2019-02-12 11:27:18,594 - [0.19322401 0.07650062 0.10272277 0.3112237  0.12059097 0.12650835
 0.20064975 0.07023515 0.25978496 0.32545637 0.09316986 0.18722927
 0.28480817 0.42090811 0.24957457 0.06973236 0.1237237  0.08249536
 0.04641089 0.03774752 0.08048422 0.12349165 0.15741027 0.12925433
 0.06845606]
INFO - 2019-02-12 11:27:19,378 - Maximum Sentence Length 407062.000000 , 90 percentile length 16420.400000 ... 
INFO - 2019-02-12 11:27:19,380 - Pos Percentage of remaining data ... 
INFO - 2019-02-12 11:27:19,383 - [0.18517694 0.06877365 0.10549744 0.30847986 0.11395504 0.12819942
 0.19719564 0.06543512 0.24905408 0.32383708 0.09014022 0.19519252
 0.30202537 0.42777654 0.23213888 0.06209659 0.1135099  0.07834409
 0.04941019 0.03360783 0.07589584 0.11551302 0.1502337  0.11662586
 0.06855108]


In [ ]:
for e in experiments :
    config = e(data, structured=True)
    print(config)
    trainer = Trainer(BasicCT, config, _type='multilabel')
    trainer.train(train_data, dev_data, save_on_metric='macro_roc_auc')

    evaluator = Evaluator(BasicCT, trainer.model.dirname, _type='multilabel')
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*300)

In [ ]:
for e in hierarchical_experiments :
    config = e(data, structured=False)
    print(config)
    trainer = Trainer(HierCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(HierCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=data.structured_columns)
dev_data = data.get_data('dev', structured=True, encodings=data.structured_columns)
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

for e in structured_experiments :
    config = e(data, structured=True, encodings=data.structured_columns)
    print(config)
    
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [13]:
from PatientVec.models.baselines.LR import LR, LDA

In [ ]:
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'multilabel'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)
# lr.get_features(n=20)


Fit BOW Classifier ...
Fit TFIDF Classifier ...


In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
print(lda.get_topics(n=10))
topics = lda.get_topics(n=10)
print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [ ]:
import os
for e in experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in hierarchical_experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in structured_experiments :
    config = e(data, structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    
for e in os.listdir('outputs/baselines/Readmission_y/baselines/') :
    filename = os.path.join('outputs/baselines/Readmission_y/baselines/', e)
    push_latest_model(filename, os.path.join('Readmission_y/baselines/', e))

In [ ]:
dataset = 'Readmission_y'
dataset_path = os.path.join('latex_evals', dataset)
output_path = os.path.join('Text-encoding-EHR/results/', dataset)
os.makedirs(output_path, exist_ok=True)

In [ ]:
dirs = os.listdir(dataset_path)
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
for d in dirs :
    subpath = os.path.join(dataset_path, d)
    output_file = os.path.join(output_path, d + '.csv')
    dfs = []
    for f in sorted(os.listdir(subpath)) :
        if os.path.isfile(os.path.join(subpath, f)) :
            d = json.load(open(os.path.join(subpath, f)))
            results = {k:d['results'][k] for k in keys_to_use}
            results['Method'] = f[:-14].replace('+', ' +').replace('_', ':')
            dfs.append(pd.DataFrame([results]))
        else :
            logging.error("%s not a file", f)
    
    dfs = pd.concat(dfs)
    dfs.to_csv(output_file, columns=['Method'] + keys_to_use, index=False)